In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

import DataProcessor

In [2]:
# EXPERIMENT_SNS = ['p5972', 'p5973', 'p5974', 'p5975', 'p5979', 'p5993']
EXPERIMENT_SNS = ['p5973', 'p5974', 'p5975', 'p5979', 'p5993']

In [3]:
for experiment_SN in EXPERIMENT_SNS:
    
    for event_number in range(9):
        
        if experiment_SN != 'p5993':
            run_number = 2
        else:
            run_number = 1
        
        print(experiment_SN, event_number)
        
        filename = f'../Data/{experiment_SN}e_updated.npz'
        
        data = np.load(filename, allow_pickle = True)
        data = data["experiment"][()]
        event = data['runs'][run_number]['events'][event_number]
        
        print(data['runs'][run_number]['events'])

        C_f = np.abs(event['rupture_speed'])
        Gamma = np.abs(event['gamma'])
        Xc = np.abs(event['X_c'])
        
        
        print(f'Cf: {C_f:.1f}m/s')
        print(f'Gamma: {Gamma:.1f}J/m2')
        print(f'Xc: {Xc:.1f}mm')

p5973 0
[{'event_time': 2402.2580000306452, 'time': array([2397.25800003, 2397.25900003, 2397.26000003, ..., 2407.25600003,
       2407.25700003, 2407.25800003]), 'normal_stress': array([4.03305769, 4.03691007, 4.0386387 , ..., 4.02609378, 4.03089691,
       4.03265023]), 'shear_stress': array([2.34561652, 2.34482273, 2.34211555, ..., 2.39859321, 2.39818446,
       2.39692269]), 'friction': array([0.58159756, 0.58084592, 0.57992698, ..., 0.59576188, 0.59495058,
       0.59437902]), 'LP_displacement': array([ 9935.23888078,  9934.51304707,  9933.72616194, ...,
       10027.71823492, 10027.42993648, 10027.83016255]), 'LP_velocity': array([-6.17875021, -4.88826445, -3.79977071, ...,  8.17225027,
        6.45867294,  6.62188737]), 'displacement': array([8227.42245322, 8227.38423789, 8227.6276837 , ..., 8312.31993376,
       8312.09913407, 8311.73679612]), 'strain': {'filename_downsampled': 'strain/strain_03.npz', 'filename': 'strain/p5973_03_100kHz_shs1.tpc5', 'time': array([2397.25806013,

KeyError: 'gamma'